# Classification Model

In [262]:
import cv2
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.layers import *
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
import pandas as pd
from tensorflow.keras.models import load_model

In [263]:
train_data_path="data/train"
test_data_path="data/val"

### Loading training data

In [264]:
X_train = []
y_train = []
image_size = 80
labels = os.listdir(train_data_path)
print("number of classes:",len(labels))
for i in labels:
    folderPath = os.path.join(f'{train_data_path}',i)
    for j in os.listdir(folderPath):
        img = cv2.imread(os.path.join(folderPath,j))
        h,w=img.shape[:2]
        if h>image_size or w>image_size:
            img = cv2.resize(img,(image_size, image_size))
        else:
            scale_factor = min(image_size/h, image_size/w)
            # Calculate the new image size
            new_height = int(h * scale_factor)
            new_width = int(w * scale_factor)

            # Calculate the amount of padding
            pad_height = image_size - new_height
            pad_width = image_size - new_width

            # Calculate the amount of padding on each side
            top = pad_height // 2
            bottom = pad_height - top
            left = pad_width // 2
            right = pad_width - left

            # Add padding to the image
            img_padded = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0))

            # Resize the image to the desired output size
            img_resized = cv2.resize(img_padded, (image_size,image_size))
            img=img_resized
        X_train.append(img)
        y_train.append(i)
        
X_train = np.array(X_train)
y_train = np.array(y_train)

number of classes: 28


In [265]:
trdata_len=len(X_train)

### Class weights calculation for weighted loss

In [266]:
class_weights={}
for i in labels:
    folderPath = os.path.join(f'{train_data_path}',i)
    class_weights[labels.index(i)]=trdata_len/len(os.listdir(folderPath))

In [267]:
#Checking the availability of GPU
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [268]:
#Shuffling data points
from sklearn.utils import shuffle
X_train,y_train=shuffle(X_train,y_train,random_state=101)
print(X_train.shape,y_train.shape)

(3386, 80, 80, 3) (3386,)


### Converting categorical labels to one hot representation

In [269]:
y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)

### Data normalization and augmentation

In [270]:
datagen = ImageDataGenerator(
    vertical_flip=True,
    rescale=1/255.,
    validation_split=0.2
)

datagen.fit(X_train)
batch_size = 32
train_generator = datagen.flow(X_train, y_train, batch_size=batch_size,subset='training')
valid_generator=datagen.flow(X_train, y_train, batch_size=batch_size,subset='validation')


### Loading test data

In [271]:
X_final_test = []
y_final_test = []
for i in labels:
    folderPath = os.path.join(f'{test_data_path}',i)
    for j in os.listdir(folderPath):
        img = cv2.imread(os.path.join(folderPath,j))
        h,w=img.shape[:2]
        if h>image_size or w>image_size:
            img = cv2.resize(img,(image_size, image_size))
        else:
            scale_factor = min(image_size/h, image_size/w)
            # Calculate the new image size
            new_height = int(h * scale_factor)
            new_width = int(w * scale_factor)

            # Calculate the amount of padding
            pad_height = image_size - new_height
            pad_width = image_size - new_width

            # Calculate the amount of padding on each side
            top = pad_height // 2
            bottom = pad_height - top
            left = pad_width // 2
            right = pad_width - left

            # Add padding to the image
            img_padded = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0))

            # Resize the image to the desired output size
            img_resized = cv2.resize(img_padded, (image_size,image_size))
            img=img_resized
        X_final_test.append(img)
        y_final_test.append(i)
        
X_final_test = np.array(X_final_test)
y_final_test = np.array(y_final_test)

y_final_test_new = []
for i in y_final_test:
    y_final_test_new.append(labels.index(i))
y_final_test = y_final_test_new
y_final_test = tf.keras.utils.to_categorical(y_final_test)

### Test data normalization

In [272]:
X_final_test=X_final_test/255.0

In [273]:
print(X_final_test.shape,y_final_test.shape)

(361, 80, 80, 3) (361, 28)


### CNN Model

In [274]:
Model = Sequential()
Model.add(Conv2D(32,(3,3),activation="relu", input_shape=(80,80,3)))
Model.add(MaxPooling2D((2,2)))
Model.add(BatchNormalization())
Model.add(Conv2D(64,(3,3),  activation="relu",padding="same"))
Model.add(MaxPooling2D((2,2)))
Model.add(BatchNormalization())
Model.add(Conv2D(128,(3,3), activation="relu",padding="same"))
Model.add(MaxPooling2D((2,2)))
Model.add(BatchNormalization())
Model.add(Conv2D(256,(3,3), activation="relu",padding="same"))
Model.add(MaxPooling2D((2,2)))
Model.add(BatchNormalization())
Model.add(Conv2D(512,(3,3), activation="relu",padding="same"))
Model.add(MaxPooling2D((2,2)))
Model.add(Flatten())
Model.add(Dense(128,activation="relu"))
Model.add(Dense(64,activation="sigmoid"))
Model.add(Dense(28,activation="softmax"))

In [275]:
Model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3) ,metrics='accuracy')
Model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_56 (Conv2D)          (None, 78, 78, 32)        896       
                                                                 
 max_pooling2d_52 (MaxPoolin  (None, 39, 39, 32)       0         
 g2D)                                                            
                                                                 
 batch_normalization_64 (Bat  (None, 39, 39, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_57 (Conv2D)          (None, 39, 39, 64)        18496     
                                                                 
 max_pooling2d_53 (MaxPoolin  (None, 19, 19, 64)       0         
 g2D)                                                            
                                                     

In [276]:
history = Model.fit(train_generator, epochs=100,validation_data=valid_generator,class_weight=class_weights)

Epoch 1/100


2023-04-09 22:02:00.930592: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


81/85 [===========================>..] - ETA: 0s - loss: 0.1125 - accuracy: 0.2077

2023-04-09 22:02:04.716638: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


85/85 [==============================] - 4s 17ms/step - loss: 0.1115 - accuracy: 0.2189 - val_loss: 3.2714 - val_accuracy: 0.0945
Epoch 2/100
85/85 [==============================] - 1s 14ms/step - loss: 0.0754 - accuracy: 0.5382 - val_loss: 2.7569 - val_accuracy: 0.2555
Epoch 3/100
85/85 [==============================] - 1s 14ms/step - loss: 0.0462 - accuracy: 0.7626 - val_loss: 2.3516 - val_accuracy: 0.3663
Epoch 4/100
85/85 [==============================] - 1s 14ms/step - loss: 0.0281 - accuracy: 0.8439 - val_loss: 2.0220 - val_accuracy: 0.4668
Epoch 5/100
85/85 [==============================] - 1s 14ms/step - loss: 0.0173 - accuracy: 0.9088 - val_loss: 0.9510 - val_accuracy: 0.8021
Epoch 6/100
85/85 [==============================] - 1s 14ms/step - loss: 0.0120 - accuracy: 0.9324 - val_loss: 0.6997 - val_accuracy: 0.8375
Epoch 7/100
85/85 [==============================] - 1s 14ms/step - loss: 0.0092 - accuracy: 0.9446 - val_loss: 0.5955 - val_accuracy: 0.8641
Epoch 8/100
85/85 

### Evaluating model

In [277]:
score_cnn = Model.evaluate(X_final_test,y_final_test, verbose = 0)

print('final Test accuracy:', score_cnn[1])

final Test accuracy: 0.9390581846237183


### Transfer learning using Resnet 

In [278]:
resnet_model=tf.keras.applications.ResNet152V2(include_top=False,weights="imagenet",input_tensor=None,input_shape=(80,80,3))

for l in resnet_model.layers:
# Freezing the model
  l.trainable = False

In [279]:
TModel = Sequential()
TModel.add(Input(shape=(80,80,3)))
TModel.add(resnet_model)
TModel.add(Flatten())
TModel.add(BatchNormalization())
TModel.add(Dense(5000,activation="relu"))
TModel.add(BatchNormalization())
TModel.add(Dense(128,activation="sigmoid"))
TModel.add(Dense(28,activation="softmax"))

In [280]:
TModel.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152v2 (Functional)    (None, 3, 3, 2048)        58331648  
                                                                 
 flatten_24 (Flatten)        (None, 18432)             0         
                                                                 
 batch_normalization_68 (Bat  (None, 18432)            73728     
 chNormalization)                                                
                                                                 
 dense_72 (Dense)            (None, 5000)              92165000  
                                                                 
 batch_normalization_69 (Bat  (None, 5000)             20000     
 chNormalization)                                                
                                                                 
 dense_73 (Dense)            (None, 128)             

In [281]:
TModel.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3) ,metrics='accuracy')

In [282]:
history = TModel.fit(train_generator, epochs=50,validation_data=valid_generator,class_weight=class_weights)

Epoch 1/50


2023-04-09 22:04:15.312926: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


85/85 [==============================] - ETA: 0s - loss: 0.0595 - accuracy: 0.6091

2023-04-09 22:04:25.622602: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


85/85 [==============================] - 13s 80ms/step - loss: 0.0595 - accuracy: 0.6091 - val_loss: 1.1184 - val_accuracy: 0.7134
Epoch 2/50
85/85 [==============================] - 5s 57ms/step - loss: 0.0251 - accuracy: 0.8320 - val_loss: 0.8867 - val_accuracy: 0.7681
Epoch 3/50
85/85 [==============================] - 5s 57ms/step - loss: 0.0154 - accuracy: 0.9014 - val_loss: 0.7919 - val_accuracy: 0.7829
Epoch 4/50
85/85 [==============================] - 5s 57ms/step - loss: 0.0104 - accuracy: 0.9299 - val_loss: 0.7680 - val_accuracy: 0.7858
Epoch 5/50
85/85 [==============================] - 5s 57ms/step - loss: 0.0072 - accuracy: 0.9505 - val_loss: 0.6861 - val_accuracy: 0.8168
Epoch 6/50
85/85 [==============================] - 5s 57ms/step - loss: 0.0044 - accuracy: 0.9708 - val_loss: 0.6376 - val_accuracy: 0.8316
Epoch 7/50
85/85 [==============================] - 5s 57ms/step - loss: 0.0029 - accuracy: 0.9797 - val_loss: 0.6669 - val_accuracy: 0.8095
Epoch 8/50
85/85 [=====

In [283]:
score_Rmodel = TModel.evaluate(X_final_test,y_final_test, verbose = 0)

#print('final Test loss:', score_cnn[0]) 
print('final Test accuracy:', score_Rmodel[1])

final Test accuracy: 0.8282548189163208


### Transfer learning using Xception model 

In [284]:
Xcep_model=tf.keras.applications.Xception(include_top=False,weights="imagenet",input_tensor=None,input_shape=(80, 80, 3))

for l in Xcep_model.layers:
# Freezing the model
  l.trainable = False

In [285]:
EModel = Sequential()
EModel.add(Input(shape=(80,80,3)))
EModel.add(Xcep_model)
EModel.add(Flatten())
EModel.add(BatchNormalization())
EModel.add(Dense(5000,activation="relu"))
EModel.add(BatchNormalization())
EModel.add(Dense(128,activation="sigmoid"))
EModel.add(Dense(28,activation="softmax"))

In [286]:
EModel.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3) ,metrics='accuracy')

In [287]:
history = EModel.fit(train_generator, epochs=50,validation_data=valid_generator,class_weight=class_weights)

Epoch 1/50


2023-04-09 22:08:32.930527: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


85/85 [==============================] - ETA: 0s - loss: 0.0703 - accuracy: 0.5090

2023-04-09 22:08:38.023363: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


85/85 [==============================] - 6s 41ms/step - loss: 0.0703 - accuracy: 0.5090 - val_loss: 1.4364 - val_accuracy: 0.6366
Epoch 2/50
85/85 [==============================] - 3s 32ms/step - loss: 0.0375 - accuracy: 0.7442 - val_loss: 1.0792 - val_accuracy: 0.7194
Epoch 3/50
85/85 [==============================] - 3s 32ms/step - loss: 0.0269 - accuracy: 0.8165 - val_loss: 0.9434 - val_accuracy: 0.7489
Epoch 4/50
85/85 [==============================] - 3s 32ms/step - loss: 0.0189 - accuracy: 0.8693 - val_loss: 0.8247 - val_accuracy: 0.7740
Epoch 5/50
85/85 [==============================] - 3s 32ms/step - loss: 0.0150 - accuracy: 0.8937 - val_loss: 0.7889 - val_accuracy: 0.7888
Epoch 6/50
85/85 [==============================] - 3s 33ms/step - loss: 0.0133 - accuracy: 0.9077 - val_loss: 0.7807 - val_accuracy: 0.7725
Epoch 7/50
85/85 [==============================] - 3s 33ms/step - loss: 0.0109 - accuracy: 0.9162 - val_loss: 0.7327 - val_accuracy: 0.7991
Epoch 8/50
85/85 [======

In [288]:
score_Emodel = EModel.evaluate(X_final_test,y_final_test, verbose = 0)

#print('final Test loss:', score_cnn[0]) 
print('final Test accuracy:', score_Rmodel[1])

final Test accuracy: 0.8282548189163208


### From the above models considering the best model based on the test accuracy

In [289]:
Best_model=Model## This is the simple CNN model without any transfer learning

### Evaluating the best model for precision,recall,f1 score

In [290]:
y_pred = Model.predict(X_final_test)
y_final_test=np.argmax(y_final_test, axis=1)
y_pred_labels = np.argmax(y_pred, axis=1)
test_acc=accuracy_score(y_pred_labels,y_final_test)
test_prec=precision_score(y_pred_labels,y_final_test,average='weighted')
test_recall=recall_score(y_pred_labels,y_final_test,average='weighted')
test_f1=f1_score(y_pred_labels,y_final_test,average='weighted')

12/12 [==============================] - 0s 3ms/step


In [297]:
Table={"data":["Testing_data"],"Accuracy":[round(test_acc,2)],"Precision":[round(test_prec,2)],"Recall":[round(test_recall,2)],"F1 Score":[round(test_f1,2)]}

In [298]:
Table=pd.DataFrame(Table)

In [299]:
Table

,data,Accuracy,Precision,Recall,F1 Score
0,Testing_data,0.94,0.94,0.94,0.94


In [300]:
Table.to_csv("Results_on_Test_data.csv")

In [295]:
Best_model.save("Best_model.h5")

## Python script to run the CNN model using OpenCV's dnn module.

In [ ]:
##To run the below code we need a saved tensorflow cnn model

Opcv_model = cv2.dnn.readNetFromTensorflow("path_to_model_graph","path_to_model_architecture")

random_img=cv2.imread("image_path")

random_img=random_img/255.0  #for normalizing the image

random_img=cv2.resize(img,(80,80))  # resizing the image

random_img=random_img[None,:]  # Extending the dimension as the model take input with size (batch,height,width,channel)

Opcv_model.setInput(random_img)

output = net.forward()

output_label = np.argmax(output)